In [1]:
import pandas as pd
import numpy as np

In [2]:
sofa_score = pd.read_csv('sofa_scores.csv')

In [3]:
all_subgroups = pd.read_csv('all_subgroups.csv')
all_subgroups = all_subgroups[['subject_id', 'hadm_id', 'subgroup']]
temp = pd.read_csv('chartevents_temp.csv')
temp = temp[temp['hadm_id'].isin(df['hadm_id'].value_counts().index)][['hadm_id', 'Temperature_final', 'charttime']]
temp = temp[temp['charttime'].isin(df['charttime'].value_counts().index)][['hadm_id', 'Temperature_final', 'charttime']]
bp = pd.read_csv('chartevents_bp.csv')
bp = bp[bp['hadm_id'].isin(all_subgroups['hadm_id'].value_counts().index)][['hadm_id', 'Combined_Blood_Pressure', 'charttime']].dropna()
def impute(heart_rate, max_rate):
    if pd.isna(heart_rate) == True:
        return max_rate
    else:
        return heart_rate
heart_rate = pd.read_csv('chartevents_hr.csv')
heart_rate['heart_rate'] = heart_rate.apply(lambda x: impute(x["Heart Rate"], x["Heart rate Alarm - High"]), axis=1)
heart_rate = heart_rate[['hadm_id', 'charttime', 'heart_rate']]
o2_state = pd.read_csv('chartevents_o2sat.csv')
o2_state = o2_state[['hadm_id', 'charttime', 'SpO2']]
respiritory_rate = pd.read_csv('chartevents_rr.csv')
respiritory_rate = respiritory_rate[['hadm_id', 'charttime', 'RespiratoryRate_combined']]
white_blood_cell = pd.read_csv('chartevents_wbc.csv')
white_blood_cell = white_blood_cell[['hadm_id', 'charttime', 'WBC_combined']]

In [4]:
df = all_subgroups.merge(bp, on = 'hadm_id', how = 'outer')

In [6]:
df = df.merge(temp, on = ['charttime', 'hadm_id'], how = 'left')

In [20]:
df = df.merge(heart_rate, on = ['charttime', 'hadm_id'], how = 'left')


In [24]:
df = df.merge(respiritory_rate, on = ['charttime', 'hadm_id'], how = 'left')

In [28]:
df = df.merge(o2_state, on = ['charttime', 'hadm_id'], how = 'left')

In [50]:
df = df.drop(columns = 'WBC_combined')

In [54]:
df = df.dropna()

In [55]:

df['date'] = pd.to_datetime(df['charttime']).dt.date
df['charttime'] = pd.to_datetime(df['charttime'])

/tmp/ipykernel_128/754730554.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = pd.to_datetime(df['charttime']).dt.date
/tmp/ipykernel_128/754730554.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['charttime'] = pd.to_datetime(df['charttime'])


In [56]:
df.isna().sum()

hadm_id                     0
Temperature_final           0
charttime                   0
subject_id                  0
subgroup                    0
Combined_Blood_Pressure     0
heart_rate                  0
RespiratoryRate_combined    0
SpO2                        0
date                        0
dtype: int64

In [57]:
#accounts for readmission 
df['hadm_number'] = df.groupby('subject_id')['hadm_id'].rank(method='dense').astype(int)

/tmp/ipykernel_128/1939590564.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['hadm_number'] = df.groupby('subject_id')['hadm_id'].rank(method='dense').astype(int)


In [58]:
df_sorted = df.sort_values(by=['subject_id', 'hadm_id', 'charttime'])

# Define a function to assign hours
def assign_hours(group):
    group['hours_in_hospital'] = (group['charttime'] - group['charttime'].iloc[0]).dt.total_seconds() / 3600
    return group

# Apply the function to each group of subject_id and hadm_id
df_with_hours = df_sorted.groupby(['subject_id', 'hadm_id']).apply(assign_hours)

# Convert hours_in_hospital to integers
df_with_hours['hours_in_hospital'] = df_with_hours['hours_in_hospital'].astype(int)

# Display the DataFrame with the new column
df_with_hours = df_with_hours.reset_index(drop = True)

In [74]:
sofa_score = sofa_score[['hadm_id', 'sofa']]

final_df = df_with_hours.merge(sofa_score, on = 'hadm_id', how = 'right')
subgroup_1 = final_df[final_df['subgroup'] == 1]

In [92]:
def groups(x):
    if 0<=x<=3:
        return 0 
    elif 4<=x<=6:
        return 1 
    else:
        return 0 

In [93]:
subgroup_1['group'] = subgroup_1['sofa'].apply(lambda x: groups(x))

/tmp/ipykernel_128/919142238.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subgroup_1['group'] = subgroup_1['sofa'].apply(lambda x: groups(x))


In [94]:
subgroup_1

,hadm_id,Temperature_final,charttime,subject_id,subgroup,Combined_Blood_Pressure,heart_rate,RespiratoryRate_combined,SpO2,date,hadm_number,hours_in_hospital,sofa,group
59,164161,37.099998,2113-08-25 12:15:00,19167.0,1.0,99.0,95.0,23.0,100.0,2113-08-25,1.0,0.0,2,0
60,164161,37.099998,2113-08-25 12:30:00,19167.0,1.0,96.0,95.0,13.0,100.0,2113-08-25,1.0,0.0,2,0
61,164161,37.099998,2113-08-25 12:45:00,19167.0,1.0,109.0,97.0,21.0,100.0,2113-08-25,1.0,0.0,2,0
62,164161,37.200001,2113-08-25 13:00:00,19167.0,1.0,100.0,96.0,20.0,100.0,2113-08-25,1.0,0.0,2,0
63,164161,37.299999,2113-08-25 14:15:00,19167.0,1.0,107.0,89.0,18.0,100.0,2113-08-25,1.0,2.0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1214728,150240,36.666698,2174-05-15 02:00:00,7241.0,1.0,150.0,90.0,12.0,100.0,2174-05-15,1.0,67.0,3,0
1214729,150240,37.000000,2174-05-15 06:00:00,7241.0,1.0,137.0,91.0,22.0,100.0,2174-05-15,1.0,71.0,3,0
1214730,150240,36.111099,2174-05-15 11:00:00,7241.0,1.0,154.0,85.0,16.0,100.0,2174-05-15,1.0,76.0,3,0
1214731,150240,35.833302,2174-05-15 19:00:00,7241.0,1.0,129.0,76.0,12.0,99.0,2174-05-15,1.0,84.0,3,0


In [95]:
X = subgroup_1.iloc[:,:-1][['Temperature_final', 'Combined_Blood_Pressure', 'heart_rate', 'RespiratoryRate_combined', 'SpO2', 'hours_in_hospital']]
y = subgroup_1.iloc[:,-1]

In [96]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [97]:
from sklearn.ensemble import RandomForestClassifier

In [83]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}
# Create a based model
rf = RandomForestClassifier()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 5, n_jobs = -1, verbose = 2)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 288 candidates, totalling 1440 fits
[CV] END bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=100; total time=   4.4s
[CV] END bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=300; total time=  13.0s
[CV] END bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=10, n_estimators=100; total time=   4.4s
[CV] END bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=10, n_estimators=100; total time=   4.2s
[CV] END bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=10, n_estimators=200; total time=   8.6s
[CV] END bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=10, n_estimators=200; total time=   8.4s
[CV] END bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=10, n_estimators=300; total time=  12.3s


/opt/conda/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV] END bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=100; total time=   4.3s
[CV] END bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=200; total time=   8.7s
[CV] END bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=1000; total time=  44.5s
[CV] END bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=10, n_estimators=1000; total time=  42.4s
[CV] END bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=100; total time=   4.4s
[CV] END bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=300; total time=  12.9s
[CV] END bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=1000; total time=  44.9s
[CV] END bootstrap=True, max_depth=80, max_features=2, min_sample

exception calling callback for <Future at 0x7fa0f12049a0 state=finished raised TerminatedWorkerError>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/joblib/externals/loky/_base.py", line 625, in _invoke_callbacks
    callback(self)
  File "/opt/conda/lib/python3.9/site-packages/joblib/parallel.py", line 359, in __call__
    self.parallel.dispatch_next()
  File "/opt/conda/lib/python3.9/site-packages/joblib/parallel.py", line 792, in dispatch_next
    if not self.dispatch_one_batch(self._original_iterator):
  File "/opt/conda/lib/python3.9/site-packages/joblib/parallel.py", line 859, in dispatch_one_batch
    self._dispatch(tasks)
  File "/opt/conda/lib/python3.9/site-packages/joblib/parallel.py", line 777, in _dispatch
    job = self._backend.apply_async(batch, callback=cb)
  File "/opt/conda/lib/python3.9/site-packages/joblib/_parallel_backends.py", line 531, in apply_async
    future = self._workers.submit(SafeFunction(func))
  File "/opt/conda/lib/

TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker.

The exit codes of the workers are {SIGKILL(-9)}

In [ ]:
grid_search.best_params_

In [98]:
rf = RandomForestClassifier(bootstrap = True,
 max_depth= 100,
 max_features= 2,
 min_samples_leaf= 3,
 min_samples_split= 8,
 n_estimators= 1000)

In [99]:
rf.fit(X_train, y_train)

RandomForestClassifier(max_depth=100, max_features=2, min_samples_leaf=3,
                       min_samples_split=8, n_estimators=1000)

In [100]:
sum(rf.predict(X_test) == y_test)/len(y_test)

0.6671805000240859

In [101]:
y_test

693139    1
819970    1
986467    0
22487     1
512243    0
         ..
767995    0
706319    0
756472    0
198429    0
772918    0
Name: group, Length: 20759, dtype: int64

In [102]:
rf.predict(X_test)

array([0, 1, 0, ..., 0, 0, 0])

In [103]:
X_test

,Temperature_final,Combined_Blood_Pressure,heart_rate,RespiratoryRate_combined,SpO2,hours_in_hospital
693139,37.777802,103.0,126.0,33.0,100.0,690.0
819970,36.444401,157.0,71.0,12.0,99.0,5.0
986467,36.400002,115.0,107.0,15.0,99.0,181.0
22487,36.888901,203.0,115.0,23.0,93.0,45.0
512243,37.000000,118.0,66.0,20.0,100.0,13.0
...,...,...,...,...,...,...
767995,37.700001,116.0,94.0,17.0,99.0,91.0
706319,37.666698,173.0,107.0,28.0,100.0,768.0
756472,35.722198,118.0,82.0,21.0,97.0,4.0
198429,36.099998,116.0,81.0,10.0,100.0,1.0
